<a href="https://colab.research.google.com/github/RicoStaedeli/NLP2025_CQG/blob/main/Training/3_Training_1_SFT_2.ipynb" target="_parent"><img src="https://colab.research.google.com/assets/colab-badge.svg" alt="Open In Colab"/></a>

# SFT Training to generate critical questions
In this training we try to use the generated scores for the evaluated SocratiQ dataset.

## Setup
First we define some constant values and also install all needed libraries



### Installation

In [ ]:
!pip install --no-deps xformers triton unsloth_zoo
!pip install sentencepiece protobuf huggingface_hub hf_transfer
!pip install --no-deps unsloth
!pip install -U transformers
!pip install -U datasets
!pip install -U accelerate
!pip install -U peft
!pip install -U trl
!pip install -U bitsandbytes

In [ ]:
from unsloth import FastLanguageModel
import shutil
import os
import torch
from datasets import load_dataset
import logging
from trl import SFTTrainer
from transformers import TrainingArguments, EarlyStoppingCallback, IntervalStrategy, DataCollatorForSeq2Seq
from unsloth import is_bfloat16_supported

### Colab
This part is only relevant when using the notebook in google colab

In [ ]:
from google.colab import userdata, drive

In [ ]:
drive.mount('/content/drive')
token = userdata.get('GITHUB')

Clone GitHub Repository to directly push generated files

In [ ]:
repo_url = f"https://{token}@github.com/RicoStaedeli/NLP2025_CQG.git"

!git clone {repo_url}

### Path Variables and Logger

In [ ]:
################################################################################
#######################   STATIC VARIABLES      ################################
################################################################################

TRAINING_NUMBER = 2
BASE_MODEL_REPO = "unsloth/Meta-Llama-3.1-8B-Instruct"
MODEL_NAME = "Meta-Llama-3.1-1B-Instruct_SFT_2"

################################################################################
#######################   PATH VARIABLES        ################################
################################################################################

train_dataset_path = "/content/drive/MyDrive/HSG/NLP/Project NLP/Data/Final/Categorised/categoriesed_filtered_train_data.json" #"/content/NLP2025_CQG/Data/Processed/example_train.json"

log_base_path = f"/content/NLP2025_CQG/Training/Logs/Traing_{TRAINING_NUMBER}/Tensorboard/"
os.makedirs(log_base_path, exist_ok=True)

log_file_path = f"/content/NLP2025_CQG/Logs/training_{TRAINING_NUMBER}.log"

model_save_path = f"/content/drive/MyDrive/HSG/NLP/Project NLP/Training/Training_{TRAINING_NUMBER}/Model/{MODEL_NAME}_finetuned/"
os.makedirs(model_save_path, exist_ok=True)

model_lora_adapter_save_path = f"/content/drive/MyDrive/HSG/NLP/Project NLP/Training/Training_{TRAINING_NUMBER}/Model/{MODEL_NAME}_lora_adapters/"
os.makedirs(model_lora_adapter_save_path, exist_ok=True)


checkpoint_dir = f"/content/drive/MyDrive/HSG/NLP/Project NLP/Training/Training_{TRAINING_NUMBER}/Checkpoints/"
os.makedirs(checkpoint_dir, exist_ok=True)


################################################################################
#######################   LOGGER                ################################
################################################################################

# Setup logger manually
logger = logging.getLogger(__name__)
logger.setLevel(logging.INFO)

# Create file handler (only if not already added)
if not logger.handlers:
    fh = logging.FileHandler(log_file_path)
    fh.setLevel(logging.INFO)
    formatter = logging.Formatter('%(asctime)s - %(levelname)s - %(message)s')
    fh.setFormatter(formatter)
    logger.addHandler(fh)

# Detect device
device = torch.device(
    "mps" if torch.backends.mps.is_available()
    else "cuda" if torch.cuda.is_available()
    else "cpu"
)

In [ ]:
logger.info("--------  Start with Training  -------------")
logger.info(f'Device selected: {device}')
logger.info(f'Model: {MODEL_NAME}')
logger.info(f'Training number: {TRAINING_NUMBER}')

## Training Parameters

In [ ]:
################################################################################
#######################   Unlsoth Parameters    ################################
################################################################################

max_seq_length = 2048 # Choose any! We auto support RoPE Scaling internally!
dtype = None # None for auto detection. Float16 for Tesla T4, V100, Bfloat16 for Ampere+
load_in_4bit = True # Use 4bit quantization to reduce memory usage. Can be False.


################################################################################
#######################   PEFT Parameters       ################################
################################################################################

r = 16 # Choose any number > 0 ! Suggested 8, 16, 32, 64, 128
target_modules = ["q_proj", "k_proj", "v_proj", "o_proj",
                  "gate_proj", "up_proj", "down_proj",]
lora_alpha = 16
lora_dropout = 0 # Supports any, but = 0 is optimized
bias = "none"    # Supports any, but = "none" is optimized
use_gradient_checkpointing = "unsloth" # True or "unsloth" for very long context
random_state = 3407
use_rslora = False  # Unsloth supports rank stabilized LoRA
loftq_config = None # And LoftQ


################################################################################
#######################   SFT Trainer Parameters   #############################
################################################################################

dataset_text_field = "text"
dataset_num_proc = 2
packing = False
per_device_train_batch_size = 38
gradient_accumulation_steps = 4
warmup_steps = 5
num_train_epochs = 1
max_steps = 40
learning_rate = 2e-4
fp16 = not is_bfloat16_supported()
bf16 = is_bfloat16_supported()
logging_steps = 1
save_strategy = IntervalStrategy.STEPS
save_steps = 10
save_total_limit = 1
optim = "adamw_8bit"
weight_decay = 0.01
lr_scheduler_type = "linear"
seed = 3407
output_dir = checkpoint_dir
report_to = "tensorboard"
logging_dir = log_base_path

################################################################################
#######################   Log Parameters            ############################
################################################################################

logger.info("------ Unlsoth Parameters ---------------")
logger.info(f"max_seq_length: {max_seq_length}")
logger.info(f"dtype: {dtype}")
logger.info(f"load_in_4bit: {load_in_4bit}")

logger.info("------ PEFT Parameters ------------------")
logger.info(f"r: {r}")
logger.info(f"target_modules: {target_modules}")
logger.info(f"lora_alpha: {lora_alpha}")
logger.info(f"lora_dropout: {lora_dropout}")
logger.info(f"bias: {bias}")
logger.info(f"use_gradient_checkpointing: {use_gradient_checkpointing}")
logger.info(f"random_state: {random_state}")
logger.info(f"use_rslora: {use_rslora}")

logger.info("------  SFT Trainer Parameters ----------")
logger.info(f"dataset_text_field: {dataset_text_field}")
logger.info(f"dataset_num_proc: {dataset_num_proc}")
logger.info(f"packing: {packing}")
logger.info(f"per_device_train_batch_size: {per_device_train_batch_size}")
logger.info(f"gradient_accumulation_steps: {gradient_accumulation_steps}")
logger.info(f"warmup_steps: {warmup_steps}")
logger.info(f"max_steps: {max_steps}")
logger.info(f"learning_rate: {learning_rate}")
logger.info(f"fp16: {fp16}")
logger.info(f"bf16: {bf16}")
logger.info(f"logging_steps: {logging_steps}")
logger.info(f"save_strategy: {save_strategy}")
logger.info(f"save_steps: {save_steps}")
logger.info(f"save_total_limit: {save_total_limit}")
logger.info(f"optim: {optim}")
logger.info(f"weight_decay: {weight_decay}")
logger.info(f"lr_scheduler_type: {lr_scheduler_type}")
logger.info(f"seed: {seed}")
logger.info(f"output_dir: {output_dir}")
logger.info(f"report_to: {report_to}")
logger.info(f"logging_dir: {logging_dir}")

### Unsloth

In [ ]:
model, tokenizer = FastLanguageModel.from_pretrained(
    model_name = BASE_MODEL_REPO,
    max_seq_length = max_seq_length,
    dtype = dtype,
    load_in_4bit = load_in_4bit,
)

We now add LoRA adapters so we only need to update 1 to 10% of all parameters!

In [ ]:
model = FastLanguageModel.get_peft_model(
    model,
    r = r,
    target_modules = target_modules,
    lora_alpha = lora_alpha,
    lora_dropout = lora_dropout,
    bias = bias,
    use_gradient_checkpointing = use_gradient_checkpointing,
    random_state = random_state,
    use_rslora = use_rslora,
    loftq_config = loftq_config,
)

## Load and preprocess dataset
The raw dataset [SocratiQ](https://github.com/NUS-IDS/eacl23_soqg/tree/main) has a label at the begining of the context. We have to remove that and also tokenize the input for the model training.

In [ ]:
dataset = load_dataset('json', data_files=train_dataset_path)

In [ ]:
print(dataset)

In [ ]:
schemas_template = {
"CauseToEffect": """'Cause to Effect' with the examples:
How strong is the generalisation that if <eventA> then <eventB>?
Are there other factors in this particular case that could have interfered with the event of‘<eventB>’?""",

"ExpertOpinion": """'Expert Opinion' with the examples:
Is <expertE> a genuine expert in <domainD>?
Is <eventA> consistent with what other experts in <domainD> say? """,

"Analogy": """'Analogy' with the examples:
Are <C1> and <C2> similar in the respect cited?
Is <eventA> true in <C1>?""",

"FearAppeal": """'Fear Appeal' with the examples:
Is <eventB> bad? Why and to whom is it bad?
Is <eventA> away to prevent <eventB>?"""
}

In [ ]:
template_chat_llama = """<|begin_of_text|><|start_header_id|>system<|end_header_id|>
You generate concise, critical, single-sentence questions for argumentative contexts, matching specified question schemas.<|eot_id|><|start_header_id|>user<|end_header_id|>
### Instruction:
Generate one critical question addressing the provided context. Ensure it matches the schema:
{}

### Context:
{}

Respond with only the generated question.<|eot_id|>

<|start_header_id|>assistant<|end_header_id|>
### Response:
{}
"""

In [ ]:
EOS_TOKEN = tokenizer.eos_token # Must add EOS_TOKEN
def formatting_prompts_func(examples):
    contexts        = examples["context"]
    questions       = examples["question"]
    schemas         = examples["schema"]

    texts = []
    for context, question, CauseToEffect, Analogy, ExpertOpinion, FearAppeal in zip(contexts, questions, CauseToEffect, Analogy, ExpertOpinion, FearAppeal):
        # Must add EOS_TOKEN, otherwise your generation will go on forever!
        schema_text = schemas_template.get(schema_key, "Schema Unknown")
        text = template_chat_llama.format(schema_text, context, question) + EOS_TOKEN
        texts.append(text)
    return { "text" : texts, }
pass

dataset = dataset.map(formatting_prompts_func, batched = True,)

In [ ]:
# Check the structure of the dataset
print(dataset['train'][0]['text'])

In [ ]:
print(dataset)

In [ ]:
train_dataset = dataset['train']

In [ ]:
print(train_dataset)

### Train the model

In [ ]:
args = TrainingArguments(
    per_device_train_batch_size=per_device_train_batch_size,
    gradient_accumulation_steps=gradient_accumulation_steps,
    warmup_steps=warmup_steps,
    max_steps=max_steps,
    # num_train_epochs=num_train_epochs,
    learning_rate=learning_rate,
    fp16=fp16,
    bf16=bf16,
    logging_steps=logging_steps,
    save_strategy=save_strategy,
    save_steps=save_steps,
    save_total_limit=save_total_limit,
    optim=optim,
    weight_decay=weight_decay,
    lr_scheduler_type=lr_scheduler_type,
    seed=seed,
    output_dir=output_dir,
    report_to=report_to,
    logging_dir=logging_dir,
)

trainer = SFTTrainer(
    model = model,
    tokenizer = tokenizer,
    train_dataset = train_dataset,
    dataset_text_field = dataset_text_field,
    max_seq_length = max_seq_length,
    data_collator = DataCollatorForSeq2Seq(tokenizer = tokenizer),
    dataset_num_proc = dataset_num_proc,
    packing = packing,
    args = args,
)

In [ ]:
# @title Show current memory stats
gpu_stats = torch.cuda.get_device_properties(0)
start_gpu_memory = round(torch.cuda.max_memory_reserved() / 1024 / 1024 / 1024, 3)
max_memory = round(gpu_stats.total_memory / 1024 / 1024 / 1024, 3)

logger.info(f"GPU  Information before Training")
logger.info(f"GPU = {gpu_stats.name}. Max memory = {max_memory} GB.")
logger.info(f"{start_gpu_memory} GB of memory reserved.")

In [ ]:
trainer_stats = trainer.train()

In [ ]:
# @title Show final memory and time stats
used_memory = round(torch.cuda.max_memory_reserved() / 1024 / 1024 / 1024, 3)
used_memory_for_lora = round(used_memory - start_gpu_memory, 3)
used_percentage = round(used_memory / max_memory * 100, 3)
lora_percentage = round(used_memory_for_lora / max_memory * 100, 3)

logger.info(f"{trainer_stats.metrics['train_runtime']} seconds used for training.")
logger.info(
    f"{round(trainer_stats.metrics['train_runtime']/60, 2)} minutes used for training."
)
logger.info(f"Peak reserved memory = {used_memory} GB.")
logger.info(f"Peak reserved memory for training = {used_memory_for_lora} GB.")
logger.info(f"Peak reserved memory % of max memory = {used_percentage} %.")
logger.info(f"Peak reserved memory for training % of max memory = {lora_percentage} %.")

## Save
Save the finetuned model

### Save Lora delta weights

In [ ]:
model.save_pretrained(model_lora_adapter_save_path)  # Local saving
tokenizer.save_pretrained(model_lora_adapter_save_path)
logger.info(f"Saved LoRA adapters to {model_lora_adapter_save_path}")

In [ ]:
token = userdata.get('HF_TOKEN')
# Push only the adapters to Hugging Face
model.push_to_hub(f"ricostaedeli/{MODEL_NAME}-lora", use_auth_token=token, private=False)

# Also save tokenizer separately
tokenizer.push_to_hub(f"ricostaedeli/{MODEL_NAME}-lora", use_auth_token=token, private=False)

### Saving merged model

Save merged model

This code is only needed if the upload failed somehow and the checkpoint has to be reloaded

In [ ]:
"""
model, tokenizer = FastLanguageModel.from_pretrained(
    model_name = model_lora_adapter_save_path,
    max_seq_length = max_seq_length,
    dtype = dtype,
    load_in_4bit = load_in_4bit,
)

token = userdata.get('HF_TOKEN2')
model.push_to_hub_merged(f"ricostaedeli/{MODEL_NAME}", tokenizer, save_method="merged_16bit", token=token, private=True)
"""

In [ ]:
# Merge to 16bit
if False:
  model.save_pretrained_merged(model_save_path, tokenizer, save_method = "merged_16bit",)
  logger.info(f"Saved merged model in 16bit to {model_save_path}")
if False:
  token = userdata.get('HF_TOKEN')
  model.push_to_hub_merged(f"ricostaedeli/{MODEL_NAME}", tokenizer, save_method="merged_16bit", token=token, private=True)

# Merge to 4bit
if False:
  model.save_pretrained_merged(model_save_path, tokenizer, save_method = "merged_4bit",)
  logger.info(f"Saved merged model in 4bit to {model_save_path}")

# Just LoRA adapters
if False:
  model.save_pretrained_merged(model_save_path, tokenizer, save_method = "lora",)
  logger.info(f"Saved LoRA adapters to {model_save_path}")

### GGUF / llama.cpp Conversion
Save model in GGUF format

Some supported quant methods (full list on [Wiki page](https://github.com/unslothai/unsloth/wiki#gguf-quantization-options)):
* `q8_0` - Fast conversion. High resource use, but generally acceptable.
* `q4_k_m` - Recommended. Uses Q6_K for half of the attention.wv and feed_forward.w2 tensors, else Q4_K.
* `q5_k_m` - Recommended. Uses Q6_K for half of the attention.wv and feed_forward.w2 tensors, else Q5_K.

In [ ]:
# Save to 8bit Q8_0
if False:
  model.save_pretrained_gguf(model_save_path, tokenizer,)
  logger.info(f"Saved merged model in 8bit Q8_0 to {model_save_path}")

# Save to 16bit GGUF
if False:
  model.save_pretrained_gguf(model_save_path, tokenizer, quantization_method = "f16")
  logger.info(f"Saved merged model in 16bit GGUF to {model_save_path}")

# Save to q4_k_m GGUF
if False:
  model.save_pretrained_gguf(model_save_path, tokenizer, quantization_method = "q4_k_m")
  logger.info(f"Saved merged model in q4_k_m GGUF to {model_save_path}")

# Save to multiple GGUF options - much faster if you want multiple!
if False:
    model.push_to_hub_gguf(
        "hf/model", # Change hf to your username!
        tokenizer,
        quantization_method = ["q4_k_m", "q8_0", "q5_k_m",],
        token = "",
    )

In [ ]:
os.chdir("NLP2025_CQG")
!ls

In [ ]:
!git config --global user.name "Rico Städeli"
!git config --global user.email "rico@yabriga.ch"


commit_message = f"Training Number: {TRAINING_NUMBER}, Training logs in Google Drive."
!git add .
!git commit -m "{commit_message}"
!git push